<a href="https://colab.research.google.com/github/enricsanz-dxc/sesdpw2g4/blob/main/SparkStream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark en Google Colab Automatico

* Instalacion Java
* Instalacion de Spark


## Instalacion Java

In [22]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

## Instalacion de Spark
Obtener automaticamente la ultima version de spark de

In [23]:
from bs4 import BeautifulSoup
import requests

In [24]:
#Obtener las versiones de spark la pagina web
url = 'https://downloads.apache.org/spark/' 
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)

In [25]:
# leer la pagina web y obtener las versiones de spark disponibles
link_files = []
for link in soup.find_all('a'):
  link_files.append(link.get('href'))
spark_link = [x for x in link_files if 'spark' in x]  
print(spark_link)

['spark-2.4.7/', 'spark-3.0.2/', 'spark-3.1.1/']


La version a usar seran las superiores a spark-2.3.4 y menores a spark-3.0.0

obtener la version y eliminar el caracter '/' del final

In [26]:
ver_spark = spark_link[2][:-1] # obtener la version y eliminar el caracter '/' del final
print(ver_spark)
ver_hadoop = 'hadoop3.2'
print(ver_hadoop)
ver_kafka = 'spark-sql-kafka-0-10_2.12:3.1.1'
print(ver_kafka)

spark-3.1.1
hadoop3.2
spark-sql-kafka-0-10_2.12:3.1.1


In [27]:
import os # libreria de manejo del sistema operativo
#instalar automaticamente la version deseadda de spark
os.system(f"wget -q https://www-us.apache.org/dist/spark/{ver_spark}/{ver_spark}-bin-{ver_hadoop}.tgz")
os.system(f"tar xf {ver_spark}-bin-{ver_hadoop}.tgz")
# instalar pyspark
!pip install -q pyspark

## Definir variables de entorno

In [28]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{ver_spark}-bin-{ver_hadoop}/"
os.environ['PYSPARK_SUBMIT_ARGS'] = ' --packages org.apache.spark:{ver_kafka} pyspark-shell '

# Cargar pyspark en el sistema

In [29]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AirPollutionReader").master("local[*]").getOrCreate()
spark

Exception: ignored

## Configuracion entorno ejecución

In [ ]:
TIME_FORMAT = '%Y-%m-%d %H:%M:%S'
TOPIC = 'airpollution'
INPUT = 'files'
SERVER= '34.65.206.2:9092'
NUM_MESSAGES=2
NUM_STATIONS=10
MESSAGE_SLEEP_TIME=5
STATION_SLEEP_TIME=1

# Lectura del Stream de Kafka

In [ ]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", SERVER) \
  .option("subscribe", TOPIC) \
  .load() 

In [ ]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [ ]:
fin = df.selectExpr("CAST(value AS STRING)") \
  .writeStream \
  .show() \
  .start()


AttributeError: ignored

In [ ]:
fin = df.selectExpr("CAST(value AS STRING)") \
    .writeStream \
    .format('json') \
    .outputMode('append')\
    .format('console') \
    .start()
fin.awaitTermination()